In [ ]:
from datetime import datetime, timedelta
from selenium import webdriver
import time
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys

In [2]:
first_part_url = "https://www.momondo.com/flight-search/"
last_part_url = "/"

In [16]:
TTT = [(datetime.today() + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(0, 30)]

In [17]:
LOS = [(datetime.today() + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(1, 6)]

In [18]:
snapshot_date = (datetime.today() - timedelta(hours=7)).strftime('%Y-%m-%d %H:%M:%S')
snapshot_date

'2025-03-02 18:35:14'

In [19]:
dep = ['LON' , 'PAR' , 'ROM' ]
back = ['LON' , 'PAR' , 'ROM' ]

In [20]:
all_urls = []

# Outer loops over the city combinations
for origin in dep:
    for destination in back:
        if origin != destination:
            # For a given city pair, iterate over all departure dates
            for t in TTT:
                # For each departure date, create return dates offset from +1 to +5 days
                for delta in range(1, 6):
                    return_date = (datetime.strptime(t, "%Y-%m-%d") + timedelta(days=delta)).strftime("%Y-%m-%d")
                    # Construct the URL including departure and return dates
                    url = f"{first_part_url}{origin}-{destination}/{t}/{return_date}{last_part_url}"
                    all_urls.append((url, t, return_date))

# Display the total count and a few sample URLs
print(f"Total URLs generated: {len(all_urls)}")
print("Sample URLs:")
for u in all_urls[:]:
    print(u)

Total URLs generated: 900
Sample URLs:
('https://www.momondo.com/flight-search/LON-PAR/2025-03-03/2025-03-04/', '2025-03-03', '2025-03-04')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-03/2025-03-05/', '2025-03-03', '2025-03-05')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-03/2025-03-06/', '2025-03-03', '2025-03-06')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-03/2025-03-07/', '2025-03-03', '2025-03-07')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-03/2025-03-08/', '2025-03-03', '2025-03-08')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-04/2025-03-05/', '2025-03-04', '2025-03-05')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-04/2025-03-06/', '2025-03-04', '2025-03-06')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-04/2025-03-07/', '2025-03-04', '2025-03-07')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-04/2025-03-08/', '2025-03-04', '2025-03-08')
('https://www.momondo.com/flight-search/LON-PAR/2025

In [21]:
driver = webdriver.Chrome()
driver.get(all_urls[0][0])

In [11]:
while True:
    elements = driver.find_elements(By.XPATH, './/*[@class="nrc6 nrc6-mod-pres-default nrc6-mod-desktop-responsive"]')
    if len(elements) >= 100:
        break
    try: #Finding "Show more results" button.
                load_more_button = WebDriverWait(driver,45).until(
                    EC.element_to_be_clickable((By.XPATH,"//div[div[contains(text(),'Show more results')]]"))
                )
                load_more_button.click()
                time.sleep(1) # small delay to allow results to load
    except:
        break
len(elements)

105

In [ ]:
from selenium.common.exceptions import StaleElementReferenceException
def handle_captcha(driver):
    """Detects CAPTCHA based on the specific structure and waits for manual resolution."""
    try:
        # Look for the CAPTCHA wrapper div
        captcha_element = driver.find_element(By.XPATH, "//div[contains(@class, 'WZTU-wrap')]")
        if captcha_element:
            print("CAPTCHA detected! Deleting cookies and refreshing the page...")            
            # Delete all cookies to enable work flow (without it - it will keep asking for captcha nonstop!!!)
            driver.delete_all_cookies()             
                # Refresh the page after deleting cookies
            driver.refresh()               
            print("Please solve the CAPTCHA manually.")
            input("Press Enter once you've completed the CAPTCHA and clicked 'Continue'.")                
    except NoSuchElementException:
        return False  # No CAPTCHA detected
with open('flight_data_momondo.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Snap Date","Dep Date","Return Date","Dep time","Dep Time","Dep Duration","From","Back time","Back Comp","Stops","Back Duration","Back Arrival","Stops","Landing","Price"])

    for url, dep_date, return_date in all_urls:
        driver.get(url)
        time.sleep(4)  
        if handle_captcha(driver):
            print("Resuming")  # If CAPTCHA is detected and solved, continue to the next URL
        while True:# Click 'Show More' button until 100 results are present
            elements = driver.find_elements(By.XPATH, './/*[@class="nrc6 nrc6-mod-pres-default nrc6-mod-desktop-responsive"]')
            if len(elements) >= 100:
                break
            try: #Finding "Show more results" button.
                load_more_button = WebDriverWait(driver,55).until(
                    EC.element_to_be_clickable((By.XPATH,"//div[div[contains(text(),'Show more results')]]"))
                )
                load_more_button.click()
                time.sleep(1) # small delay to allow results to load
            except:
                break #exit loop if the button is missing or unclickable.


        # Extracting flight details

        for item in driver.find_elements(By.XPATH, './/*[@class="nrc6 nrc6-mod-pres-default nrc6-mod-desktop-responsive"]'):
            retry_count = 3  # Number of retry attempts
            for attempt in range(retry_count):
                try:
                    data = item.find_element(By.XPATH, './/*[@class="nrc6-inner"]').text
                    break  # If no exception, break out of retry loop
                except StaleElementReferenceException:
                    if attempt < retry_count - 1:  # If it's not the last attempt, retry
                        print("StaleElementReferenceException detected. Retrying...")
                        time.sleep(1)  # Wait before retrying
                    else:
                        print("Failed to find element after several retries.")
            data_parts = data.split("\n")
            #Removing "Throwaway ticketing hack" | bags | cabins if it exists.
            data_parts = [part for part in data_parts if part.strip() not in ("Save","+1",+1,0,"0",1,"1","-","Share","Ad","Best","Cheapest","View Deal","Mix & Match")
                          and "with Booking.com" not in part and "sites" not in part and "Operated by" not in part and "As low as" not in part]

            #image_element = driver.find_element(By.XPATH, './/*[@class="max-w-none bg-transparent rounded-100 last:self-end h-icon-large w-icon-large"]')
            #dep_back_companies = image_element.get_attribute("title")
            #companies_parts = dep_back_companies.split("\n")
            #price = item.find_element(By.XPATH, './/*[@class="Fxw9-result-item-container"]')
            writer.writerow([snapshot_date,dep_date, return_date] + data_parts)

# Close WebDriver
driver.quit()

CAPTCHA detected! Deleting cookies and refreshing the page...
Please solve the CAPTCHA manually.
CAPTCHA detected! Deleting cookies and refreshing the page...
Please solve the CAPTCHA manually.
CAPTCHA detected! Deleting cookies and refreshing the page...
Please solve the CAPTCHA manually.
CAPTCHA detected! Deleting cookies and refreshing the page...
Please solve the CAPTCHA manually.
StaleElementReferenceException detected. Retrying...
StaleElementReferenceException detected. Retrying...
Failed to find element after several retries.
StaleElementReferenceException detected. Retrying...
StaleElementReferenceException detected. Retrying...
Failed to find element after several retries.
StaleElementReferenceException detected. Retrying...
StaleElementReferenceException detected. Retrying...
Failed to find element after several retries.
StaleElementReferenceException detected. Retrying...
StaleElementReferenceException detected. Retrying...
Failed to find element after several retries.
Stal